# Identify and extract interesting data-set for vignettes

Here I'd like to pick up ideally examples that has enough power for susie to detect 3 simulated signals, at non-trivial yet reasonable number of iterations, size of sets and purity levels, for illustrating how susie works. In other words this is meant to pick up good susie show cases, not edge cases.

In [1]:
%cd ~/GIT/github/mvarbvs/dsc/susie_comparison/fit_susie

/home/gaow/GIT/github/mvarbvs/dsc/susie_comparison/fit_susie

In [ ]:
[1]
num_causal = 3
input: glob.glob(f'liter_data_*_summarize_ld_1_lm_less_{num_causal}_fit_susie_*.rds')[:200], group_by = 'single' 
R: expand = '${ }'
  res = readRDS(${_input:r})
  for (r in 1:2) {
  	cs = susieR::susie_get_CS(res$posterior$alpha[[r]])$cs[1:${num_causal}]
  	if (length(cs[[${num_causal}]]) < 20) {
  		print(${_input:r})
		print(r)
  		print(res$posterior$niter[r])
		print(cs)
		print("========")
  	}
  }

I ended up with `liter_data_65_summarize_ld_1_lm_less_3_fit_susie_7.rds` the first data-set, which seems interesting:

```
[[1]]
[1] 773 777

[[2]]
 [1] 360 361 362 365 368 372 373 374 379 381 383 384 386 387 388 389 391 392 396
[20] 397 398 399 400 401 403 404 405 407 408 415

[[3]]
[1] 653
```

takes 6 iterations to complete.

In [2]:
dat = readRDS('liter_data_65_summarize_ld_1_lm_less_3_fit_susie_7.rds')
names(dat)

[1] "posterior" "fitted"    "DSC_DEBUG"

From `dat$DSC_DEBUG$script` I load the dataset of interest:

In [6]:
input <- dscrutils::load_inputs(c('../lm_less/liter_data_65_summarize_ld_1_lm_less_3.pkl'), dscrutils::read_dsc)

In [7]:
names(input)

[1] "data"      "V"         "N"         "DSC_DEBUG"

Now I compute summary stats and save it.

In [9]:
library(abind)
mm_regression = function(X, Y, Z=NULL) {
  if (!is.null(Z)) {
      Z = as.matrix(Z)
  }
  reg = lapply(seq_len(ncol(Y)), function (i) simplify2array(susieR:::univariate_regression(X, Y[,i], Z)))
  reg = do.call(abind, c(reg, list(along=0)))
  # return array: out[1,,] is betahat, out[2,,] is shat
  return(aperm(reg, c(3,2,1)))
}
sumstats = mm_regression(as.matrix(input$data$X), as.matrix(input$data$Y))

In [11]:
saveRDS(list(data=input$data, sumstats = sumstats), '~/GIT/software/susieR/inst/data/N3finemapping.rds')